In [1]:
from google.colab import files, drive
drive.mount('/content/gdrive')
# files.upload()

Mounted at /content/gdrive


In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 17.8MB/s 
     |████████████████████████████████| 3.3MB 44.6MB/s 
     |████████████████████████████████| 901kB 49.8MB/s 


In [3]:
%matplotlib inline
#Importing Transformers
import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, AdamW, Trainer, TrainingArguments
from torch.utils.data import DataLoader

#Importing Pandas
import pandas as pd
import glob

#Importing Numpy
import numpy as np

import torch
import json
import nltk
import random
import seaborn as sns
import re
import string
import matplotlib.pyplot as plt
import operator
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from sklearn.model_selection import train_test_split

In [4]:
transformers.__version__

'4.5.1'

In [5]:
config = AutoConfig.from_pretrained("ProsusAI/finbert")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", config=config)

In [13]:
# your path to folder containing excel files
datapath = "/content/gdrive/MyDrive/StockMarket/Shared Task/News Article/"
list_df = []

# set all .xls files in your folder to list
allfiles = glob.glob(datapath + "*.xlsx")
# for loop to aquire all excel files in folder
for excelfiles in allfiles:
  raw_excel = pd.read_excel(excelfiles)
  raw_excel = raw_excel.loc[:, ~raw_excel.columns.str.contains('^Unnamed')]
  list_df.append(raw_excel)
merged = pd.concat(list_df)
merged = merged.reset_index(drop=True)
merged = merged[merged['Sentiment'].notna()]
merged = merged[merged['Sentiment'] != 'Only title']
merged['Sentiment'] = merged['Sentiment'].apply(lambda x: x.lower())

In [35]:
len(merged)

405

In [15]:
merged['Sentiment'].value_counts()

positive    219
neutral     110
negative     75
poisitve      1
Name: Sentiment, dtype: int64

In [16]:
def to_sentiment(sentiment):
  if sentiment == "positive":
    return 1
  elif sentiment == "neutral":
    return 0
  else:
    return 2

In [17]:
merged['Sentiment'] = merged.Sentiment.apply(to_sentiment)

In [18]:
train_texts = merged['Title'].tolist()
train_labels = merged['Sentiment'].tolist()

In [19]:
X_train, X_test, y_train, y_test = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

In [20]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

243
81
81


In [21]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
val_encodings = tokenizer(X_val, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [22]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, y_train)
val_dataset = CustomDataset(val_encodings, y_val)
test_dataset = CustomDataset(test_encodings, y_test)

In [23]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

Step,Training Loss
10,2.946900
20,2.621100
30,2.796900
40,2.187700
50,1.729500
60,1.520600
70,1.162800
80,1.061500
90,0.938200
100,0.947800


TrainOutput(global_step=620, training_loss=0.3749859888970311, metrics={'train_runtime': 105.8119, 'train_samples_per_second': 5.859, 'total_flos': 98969651106120.0, 'epoch': 20.0, 'init_mem_cpu_alloc_delta': 2366021632, 'init_mem_gpu_alloc_delta': 439075328, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 14843904, 'train_mem_gpu_alloc_delta': 1317807616, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 179605504})

In [24]:
trainer.save_model()

In [25]:
resampled_model = AutoModelForSequenceClassification.from_pretrained("./results", config=config)

In [27]:
text_sample = "Hey! How are you doing?"
sample_encodings = tokenizer(text_sample, truncation=True, padding=True,
                              return_tensors="pt")

In [28]:
output = resampled_model(**sample_encodings)

In [29]:
logit = output.logits[0]
softmax_score = torch.nn.functional.softmax(logit,dim=-1)

In [30]:
torch.argmax(softmax_score)

tensor(0)

In [47]:
Mahindra_news = pd.read_excel('articles_sbi_again3.xlsx')
Mahindra_news = Mahindra_news.loc[:, ~Mahindra_news.columns.str.contains('^Unnamed')]

In [48]:
Mahindra_news_mod = Mahindra_news.copy(deep=True)

In [49]:
Mahindra_news_mod = Mahindra_news_mod.dropna()

In [50]:
Mahindra_news_mod['title_encoded'] = Mahindra_news_mod['Title'].apply(lambda x: tokenizer.encode_plus(x,
                                                                                            truncation=True,
                                                                                            padding=True,
                                                                                            return_tensors="pt"))

In [51]:
classes=['positive','negative','neutral']

In [52]:
Mahindra_news_mod['sentiment'] = Mahindra_news_mod['title_encoded'].apply(lambda x: dict(zip(classes, torch.softmax(resampled_model(**x)[0], dim=1).tolist()[0])))

In [53]:
Mahindra_news_mod.head()

,Date,Media,Title,Article,Summary,title_encoded,sentiment
0,"Mar 6, 2020",Reuters,"India sets out Yes Bank rescue, State Bank of ...",NEW DELHI/MUMBAI (Reuters) - India laid out a ...,NEW DELHI/MUMBAI (Reuters) - India laid out a ...,"[input_ids, token_type_ids, attention_mask]","{'positive': 7.967252895468846e-05, 'negative'..."
1,"Mar 6, 2020",The Hindu,"Yes Bank | In RBI bailout, SBI to pick up 49% ...","PSU bank to invest ₹2,450 crore; reconstructio...","According to the draft resolution plan, the au...","[input_ids, token_type_ids, attention_mask]","{'positive': 7.630619074916467e-05, 'negative'..."
2,"Mar 6, 2020",ThePrint,State Bank of India to buy 49% stake in Yes Bank,Text Size: A- A+\n\nNew Delhi: State Bank of I...,Text Size: A- A+New Delhi: State Bank of India...,"[input_ids, token_type_ids, attention_mask]","{'positive': 8.917888044379652e-05, 'negative'..."
3,"Feb 26, 2020",Payments Journal,"State Bank of India IPO: Ready for Take Off, B...",The biggest news in global credit cards right ...,The biggest news in global credit cards right ...,"[input_ids, token_type_ids, attention_mask]","{'positive': 0.22336287796497345, 'negative': ..."
4,"Mar 31, 2020",Mint,SBI cautions against fake UPI ID for collectin...,Country's largest lender State Bank of India (...,"""Beware of Fake UPI ID being circulating on th...","[input_ids, token_type_ids, attention_mask]","{'positive': 0.00024840154219418764, 'negative..."


In [54]:
Mahindra_news_mod['sentiment_news'] = Mahindra_news_mod['sentiment'].apply(lambda x: max(x.items(), key=operator.itemgetter(1))[0])

In [55]:
Mahindra_news_mod['sentiment_news'].value_counts()

negative    127
neutral      44
positive     12
Name: sentiment_news, dtype: int64

In [56]:
Mahindra_news_mod.to_excel("sbi_sentiment_check.xlsx")